In [1]:
import boto3
import io
import pandas as pd



client = boto3.client('s3') #low-level functional API

s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY) #high-level object-oriented API
obj = client.get_object(Bucket=MY_BUCKET, Key=OBJ_KEY) #subsitute this for your s3 bucket name. 

print(obj['Body'])
#response = obj['Body'].read()
#print(response)

In [2]:
import io
df = pd.read_csv(io.BytesIO(obj['Body'].read()))
print(df.head())

        id                        teacher_id teacher_prefix school_state  \
0  p036502  484aaf11257089a66cfedc9461c6bd0a            Ms.           NV   
1  p039565  df72a3ba8089423fa8a94be88060f6ed           Mrs.           GA   
2  p233823  a9b876a9252e08a55e3d894150f75ba3            Ms.           UT   
3  p185307  525fdbb6ec7f538a48beebaa0a51b24f            Mr.           NC   
4  p013780  a63b5547a7239eae4c1872670848e61a            Mr.           CA   

  project_submitted_datetime project_grade_category  \
0           2016-11-18 14:45          Grades PreK-2   
1           2017-04-26 15:57             Grades 3-5   
2           2017-01-01 22:57             Grades 3-5   
3           2016-08-12 15:42             Grades 3-5   
4            2016-08-06 9:09             Grades 6-8   

            project_subject_categories  \
0                  Literacy & Language   
1    Music & The Arts, Health & Sports   
2  Math & Science, Literacy & Language   
3                      Health & Sports   
4 

In [3]:

OBJ_KEY = 'resources_sub.csv'

res_obj = client.get_object(Bucket=MY_BUCKET, Key=OBJ_KEY) #subsitute this for your s3 bucket name. 
df_res = pd.read_csv(io.BytesIO(res_obj['Body'].read()))
print(df_res.head())

        id                                        description  quantity  \
0  p233245  LC652 - Lakeshore Double-Space Mobile Drying Rack         1   
1  p069063        Bouncy Bands for Desks (Blue support pipes)         3   
2  p069063  Cory Stories: A Kid's Book About Living With Adhd         1   
3  p069063  Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...         2   
4  p069063  EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...         3   

    price  
0  149.00  
1   14.95  
2    8.45  
3   13.59  
4   24.95  


In [4]:
df_train = pd.merge(df,df_res, on = 'id')
print(df_train.head())

        id                        teacher_id teacher_prefix school_state  \
0  p232007  e7a8f866e3174a77ffe37323f032a8ac           Mrs.           FL   
1  p232007  e7a8f866e3174a77ffe37323f032a8ac           Mrs.           FL   
2  p232007  e7a8f866e3174a77ffe37323f032a8ac           Mrs.           FL   
3  p232007  e7a8f866e3174a77ffe37323f032a8ac           Mrs.           FL   
4  p232007  e7a8f866e3174a77ffe37323f032a8ac           Mrs.           FL   

  project_submitted_datetime project_grade_category  \
0            2016-04-27 9:58          Grades PreK-2   
1            2016-04-27 9:58          Grades PreK-2   
2            2016-04-27 9:58          Grades PreK-2   
3            2016-04-27 9:58          Grades PreK-2   
4            2016-04-27 9:58          Grades PreK-2   

              project_subject_categories  \
0  Applied Learning, Literacy & Language   
1  Applied Learning, Literacy & Language   
2  Applied Learning, Literacy & Language   
3  Applied Learning, Literacy & Lang

In [2]:
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def text_processing(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    '''
    
    # split into words by white space
    words = text.split()
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in words]
    return [word for word in stripped if word.lower() not in stopwords.words('english')]


In [6]:
%%timeit

# Timing apply on the Haversine function
df['essay_1_wrd'] = df['project_essay_1'].apply(text_processing)

13.8 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
X = df['project_essay_1']
bow_transformer = CountVectorizer(analyzer=text_processing).fit(X)